In [76]:
using MatrixNetworks
using Distributions
using Gadfly
using Clustering
using BenchmarkTools, Compat
using DelimitedFiles
using SparseArrays
using LinearAlgebra
using SphericalGraphEmbedding
using Random

In [77]:
dataset = "PolBlogs"
dataset = "PowerEU"
dataset = "Facebook"
dataset = "PowerUS"
dataset = "HEP"
#dataset = "Internet"
# dataset = "CondMat2003"
#dataset = "Pokec"

# Initialization
A = 0
r = 0
dim0 = 0
N = 0

# nb iterations projected power method
n_it_PPM = 30000

# relative objective variation
t = 1e-8

# nb of times vector partition is repeated
n_it_vec_part = 10 

# nb updates vec part
n_updates = 50 

# shape of embedding
shape = "Ellipsoidal"

# Loading data

if dataset == "PolBlogs"

    A0 = readdlm("data/PolBlogs/PolBlogsAdj.txt")
    lab = readdlm("data/PolBlogs/LabelsPolBlogs.txt")

    dim0 = size(A0)
    A  = zeros(Int64 ,dim0[1],dim0[1])

    for i = 1:dim0[1]
        for j = 1:dim0[1]
            A[i,j] = Int64(A0[i,j])
        end
    end
    A = A+A'
    A = sparse(A)

    A,_ = largest_component(A)

    A0 = 0
    N = size(A,2)

    # number of columns of initial guess
    d0 = 20

    dim_embed_spectral = 2

    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "PowerEU"

    list = readdlm("data/PowerEU/ElistPowerEU.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j])
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)
    N = size(A,2)
    
    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50

    dim_embed_spectral = 50

    # number of centroids thrown in embedding
    n_clusters = 100


elseif dataset == "Facebook"

    list = readdlm("data/Facebook/facebook-gender_edges.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # here start at 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) # here start at 0

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50

    dim_embed_spectral = 8

    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "PowerUS"

    list = readdlm("data/PowerUS/ElistPowerUS.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j])
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50

    dim_embed_spectral = 80

    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "HEP"

    list = readdlm("data/HEP/hep-year_edges.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) 
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50

    dim_embed_spectral = 40


    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Wikivote"

    list = readdlm("data/Wikivote/wiki-Vote.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) 
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50

    dim_embed_spectral = 3

    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Internet"

    list = readdlm("data/Internet/InternetEList.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50

    dim_embed_spectral = 18

    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "CondMat2003"

    list = readdlm("data/CondMat2003/CondMat2003EList.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    p = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50

    dim_embed_spectral = 80

    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Pokec"

    list = readdlm("data/Pokec/soc-pokec-relationships.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50
    dim_embed_spectral = 50

    # number of centroids thrown in embedding
    n_clusters = 100

end

100

In [78]:
community = zeros(Int64,N,1)
x_embed,community,singular =  @time sphere_embed_cluster(A, n_it_PPM, t, n_clusters, n_it_vec_part, n_updates, shape, d0)
# run twice to get timings without compilation time
println(dataset)

 ------- Acc Projected Power Iteration -------
The iteration has become stationary after 111 iterations
 95.444524 seconds (3.39 k allocations: 10.174 GiB, 6.27% gc time)
 -------------- Clustering ------- 
dimension of embedding used for clustering: 50
Number of updates: 37
Number of communities: 11
Modularity: 0.6201242829562491
 -------------------------------------------- 
The first 5 squared singular values divided by N : 
[0.527310896378853, 0.2491721352969269, 0.1603629710122631, 0.04196229216936058, 0.02108053602060397]
 -------------------------------------------- 
242.666619 seconds (21.95 M allocations: 26.686 GiB, 4.82% gc time)
HEP


In [79]:
tol = 1e-06
it_max = 1000
# spectral embedding
community = zeros(Int64,N,1)
x_embed, community, eigenvalues = @time spectral_embed_cluster(A, it_max, tol, n_clusters, n_rep_vec_part, n_updates, dim_embed_spectral)
println(dataset)

spectral embedding computed
Number of updates: 21
Number of communities: 28
Modularity: 0.5303062192918795
922.557853 seconds (21.27 M allocations: 549.647 GiB, 9.80% gc time)
HEP
